# Imports

In [ ]:
import os
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Model

## Simple Model

In [ ]:
def simple_model(input_shape=None, classes = None):
    x_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x_input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Conv2D(128,(3,3),activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Dense(classes, activation='softmax')(x)
    model  = tf.keras.models.Model(inputs = x_input, outputs = x)
    
    return model

## VGG19

In [ ]:
# VGG19 
def define_model(input_shape, classes):
    model =  VGG19(include_top=False, input_shape=input_shape)
    # mark loaded layers as non trainable
    for layer in model.layers:
        layer.trainable = False

    # new classifier layer
    flat = Flatten()(model.layers[-1].output)
    dense = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat)
    output = Dense(classes, activation='softmax')(dense)

    #define the model
    model = Model(inputs=model.inputs, outputs=output)

    return model



# Data Preproccessing

In [ ]:
num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32

#train_data_dir = './dataset/train'
#validation_data_dir = './dataset/validation'

In [ ]:
"""
train_data_generator = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

validation_data_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_data_generator.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)
                            
"""

In [ ]:
dataset = h5py.File('./dataset/facial_emotion_dataset.h5', 'r')
train_x = np.array(dataset['data_pixel'][:])
train_y = np.array(dataset['data_label'][:])

train_y = train_y.T

print(train_x.shape)
print(train_y.shape)

train_x = train_x[:,:,:,tf.newaxis]
train_x = train_x / 255.0

print(train_x.shape)
print(train_y.shape)
train_y = to_categorical(train_y)

print(train_x.shape)
print(train_y.shape)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)

In [ ]:
checkpoint = ModelCheckpoint('facial_simple.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
tensorboard = TensorBoard(
                log_dir=logdir,
                histogram_freq=10,
                profile_batch=0,
                write_graph=True,
                write_images=False,
                update_freq="epoch")

callbacks = [earlystop,checkpoint,reduce_lr, tensorboard]

In [ ]:
input_shape = (img_rows,img_cols, 1)
model = simple_model(input_shape=input_shape, classes = 7)

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])


In [ ]:
model.fit(train_ds, epochs=50, callbacks = callbacks)

In [ ]:
model.save()